In [1]:
import pandas as pd
import gc

In [2]:
file_path_action = r'C:\Users\snowball\Desktop\py_workspace\hack\train-click.json'

In [3]:
raw = []
ct = 0

# 目前預設讀100萬筆, 實際整體資料大於3000萬, 請依記憶體量力而為
limit_cnt =  1000

with open(file_path_action) as f:
    for line in f:
        raw.append(line)
        ct = ct + 1
        
        if ct == limit_cnt:
            print(len(raw), ' size finished!')
            break

1000  size finished!


In [4]:
output = ','.join(raw)


# In[5]:

del raw
gc.collect()


# In[6]:

output = output.replace('\n', '')


# In[7]:

gc.collect()


# In[8]:

output = pd.read_json('[' + output + ']')

In [6]:
def train_click_preprocess(train_click_df):
    # 取得是否點擊的 target
    def get_target(train_click_df):
        df = train_click_df.copy()
        jobno_order_lists = []
        dfs = [] 
        joblist = df['joblist'].tolist()
        jobno = df['jobno'].tolist()
        for i in range(len(jobno)):
            df = pd.DataFrame(joblist[i], columns=['all_jobno'])
            df['index'] = i 
            df['jobno'] = jobno[i]
            df['target'] = (df['jobno'] == df['all_jobno']).astype(int)
            dfs.append(df)
        del df
        gc.collect()
        target_df = pd.concat(dfs).reset_index(drop=True).drop('jobno', axis=1)
        target_df.columns = ['jobno', 'index', 'target']
        target_df = target_df.merge(train_click_df.reset_index().drop('jobno', axis=1), on='index', how='left')
        return target_df

    # 把 query string 拆開轉成 dataframe
    def queryProcessor(querystring):
        qs = querystring.split('&')
        d = {}
        for q in qs:
            d[q.split('=')[0]] =  q.split('=')[1]
        df = pd.DataFrame([d.values()], columns=d.keys())
        return df
    
    # 取得工作清單數量、排序
    def jobno_order(train_click_df):
        df = train_click_df.copy()
        jobno_order_lists = []
        joblist = df['joblist'].tolist()
        jobno = df['jobno'].tolist()
        for i in range(len(jobno)):
            jobno_order_list = []
            jobno_order_list.append(len(joblist[i]))
            jobno_order_list.append(joblist[i].index(jobno[i]))
            jobno_order_lists.append(jobno_order_list)
        df = pd.concat([df, pd.DataFrame(jobno_order_lists, columns=['joblist_count', 'job_order_in_list']).apply(pd.to_numeric,downcast='unsigned')], axis=1)
        return df
    # 處理多選欄位變成 dummies variable 
    def multiple_values_to_dummies(df, colName):
        try:
            return pd.get_dummies(df[colName].str.split('%2C') \
                .apply(lambda x: pd.Series(x)).stack() \
                .reset_index(level=1, drop=True).to_frame(colName)) \
                .reset_index().groupby('index').sum().apply(pd.to_numeric,downcast='unsigned') \
                .join(df.drop(colName, axis=1), how='left') \
                .reset_index(drop=True)
        except:
            return df
    
    train_click_df = train_click_df.pipe(get_target)
    query_df = pd.concat(train_click_df['querystring'].apply(queryProcessor).tolist()).reset_index(drop=True)
    train_click_df = pd.concat([query_df, train_click_df.drop('querystring', axis=1)], axis=1)
    # order 可能做不出來
    # 工作清單數量都是 20 筆，可能沒用
    train_click_df = train_click_df.pipe(jobno_order)
    train_click_df['job_order_in_list'] = train_click_df['job_order_in_list'] + 1
    
    train_click_df.fillna('null', inplace=True)
    for col in train_click_df[query_df.columns]:
        train_click_df = multiple_values_to_dummies(train_click_df, col)
    del query_df
    gc.collect()
    train_click_df['target'] = train_click_df['target'].apply(pd.to_numeric,downcast='unsigned')
    return train_click_df

In [7]:
rs = train_click_preprocess(output)

In [9]:
rs.shape

(20000, 1105)